In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm

training_data = np.load("training_data.npy", allow_pickle = True)

In [5]:
print(len(training_data))

24946


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)   
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512,2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        
        if self._to_linear is None:
            print(x[0].shape)
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x
    
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = (self.fc2(x))
        return F.softmax(x, dim = 1)


net = Net()

torch.Size([128, 2, 2])


In [18]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr = 0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in  training_data]).view(-1, 50, 50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X) * VAL_PCT)
print(val_size)

2494


In [19]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_y))

22452
2494


In [24]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
        batch_X = train_X[i:i + BATCH_SIZE].view(-1,1,50,50)
        batch_y = train_y[i:i + BATCH_SIZE]
        
        net.zero_grad()
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
print(loss)


 69%|██████▉   | 155/225 [00:35<00:15,  4.55it/s]


100%|██████████| 225/225 [00:50<00:00,  4.45it/s]


tensor(0.2038, grad_fn=<MseLossBackward>)


In [25]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0] 
        
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total+=1

print("Accuracy:", round(correct/total, 3))
    


100%|██████████| 2494/2494 [00:05<00:00, 492.89it/s]

Accuracy: 0.613


In [28]:
net(test_X[i].view(-1, 1, 50, 50))

tensor([[0.3738, 0.6262]], grad_fn=<SoftmaxBackward>)